In [1]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime, timedelta
import datetime 
from pymongo import MongoClient
import pprint
import csv
hostip='192.168.1.105' # ip address

In [2]:
client = MongoClient(hostip, 27017)  # connect to MongoClient
database_names = client.list_database_names() # find each database names
for database_name in database_names:
    print(database_name)

admin
config
fit3182
fit3182DD
fit3182_db
local


In [3]:
db = client.fit3182
climate_db= db["climate_db"]

In [4]:
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8') # encode it as 'utf-8'
        producer_instance.send(topic_name, key=key_bytes, value=data) # send the encoding data
        producer_instance.flush() 
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [5]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],  #create kafka producer
                                  value_serializer=lambda x:dumps(x).encode('ascii'), #for encoding
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [6]:
def collectHotSpot(filename,n):
  
    
    with open(filename, 'r') as csvfile:
        # convert csv to dict
        data = csv.DictReader(csvfile)
        prep_data = []
        for row in data:
           
            row['publisher']= n #which publisher
            row['published_time']= datetime.datetime.now().time().strftime("%H:%M:%S") # getting the current time from Pacific Time or whateve
          
            prep_data.append(row) #append each processed row
        return prep_data

In [7]:

t=3 # it will send a hotspot in 3 sec which is 8 hours a report in the simulation
if __name__ == '__main__':
    topic = 'Terra_HotSpot' #topic for terra streaming 
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    terHPSet = collectHotSpot('hotspot_TERRA_streaming.csv',3)  #streaming from terra satellite
   
    random.seed(32939375)
    while True:
        data = terHPSet[(random.randrange(0,len(terHPSet)))] # choose from a random hotspot
        
        print(type(data))
        publish_message(producer, topic, 'jsondata', data) # send as json object
        sleep(t)

Publishing records..
<class 'dict'>
Message published successfully. Data: {'latitude': '-36.1002', 'longitude': '147.6494', 'confidence': '50', 'surface_temperature_celcius': '39', 'publisher': 3, 'published_time': '13:35:57'}
<class 'dict'>
Message published successfully. Data: {'latitude': '-38.0229', 'longitude': '143.749', 'confidence': '73', 'surface_temperature_celcius': '47', 'publisher': 3, 'published_time': '13:35:57'}
<class 'dict'>
Message published successfully. Data: {'latitude': '-36.1271', 'longitude': '145.1541', 'confidence': '75', 'surface_temperature_celcius': '48', 'publisher': 3, 'published_time': '13:35:57'}
<class 'dict'>
Message published successfully. Data: {'latitude': '-36.6823', 'longitude': '142.5024', 'confidence': '93', 'surface_temperature_celcius': '71', 'publisher': 3, 'published_time': '13:35:57'}
<class 'dict'>
Message published successfully. Data: {'latitude': '-38.2481', 'longitude': '142.8381', 'confidence': '68', 'surface_temperature_celcius': '4

KeyboardInterrupt: 